In [1]:
%%capture
!pip install spotipy
import networkx as nx
import pandas as pd
import spotipy
from spotipy . oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import operator
from collections import Counter
import community as community_louvain
import itertools

In [2]:
CLIENT_ID = "fe233373eb024c3a97f1eeca78dba3c8"
CLIENT_SECRET = "34f0386560eb4dcb88e317f81b11b961"

auth_manager = SpotifyClientCredentials (client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
sp = spotipy . Spotify ( auth_manager = auth_manager )

In [ ]:
def num_common_nodes(*arg):
    """
    Return the number of common nodes between a set of graphs.

    :param arg: (an undetermined number of) networkx graphs.
    :return: an integer, number of common nodes.
    """
    common_nodes = set(arg[0].nodes())
    for g in arg[1:]:
        common_nodes.intersection_update(set(g.nodes()))
    return len(common_nodes)

def get_degree_distribution(g: nx.Graph) -> dict:
    """
    Get the degree distribution of the graph.

    :param g: networkx graph.
    :return: dictionary with degree distribution (keys are degrees, values are number of occurrences).
    """
    degrees = [g.degree(n) for n in g.nodes()]
    degree_distribution = Counter(degrees)
    return degree_distribution

def get_k_most_central(g: nx.Graph, metric: str, num_nodes: int) -> list:
    """
    Get the k most central nodes in the graph.

    :param g: networkx graph.
    :param metric: centrality metric. Can be (at least) 'degree', 'betweenness', 'closeness' or 'eigenvector'.
    :param num_nodes: number of nodes to return.
    :return: list with the top num_nodes nodes with the specified centrality.
    """
    if metric == 'degree':
        centrality = nx.degree_centrality(g)
    elif metric == 'betweenness':
        centrality = nx.betweenness_centrality(g)
    elif metric == 'closeness':
        centrality = nx.closeness_centrality(g)
    elif metric == 'eigenvector':
        centrality = nx.eigenvector_centrality(g)
    else:
        raise ValueError(f"Unknown metric: {metric}")

    sorted_centrality = sorted(centrality.items(), key=operator.itemgetter(1), reverse=True)
    return [node for node, centrality in sorted_centrality[:num_nodes]]

def find_cliques(g: nx.Graph, min_size_clique: int) -> tuple:
    """
    Find cliques in the graph g with size at least min_size_clique.

    :param g: networkx graph.
    :param min_size_clique: minimum size of the cliques to find.
    :return: two-element tuple, list of cliques (each clique is a list of nodes) and
        list of nodes in any of the cliques.
    """
    cliques = list(nx.find_cliques(g))
    cliques = [c for c in cliques if len(c) >= min_size_clique]
    nodes_in_cliques = set(itertools.chain.from_iterable(cliques))
    return cliques, list(nodes_in_cliques)

def detect_communities(g: nx.Graph, method: str) -> tuple:
    """
    Detect communities in the graph g using the specified method.

    :param g: a networkx graph.
    :param method: string with the name of the method to use. Can be (at least) 'givarn-newman' or 'louvain'.
    :return: two-element tuple, list of communities (each community is a list of nodes) and modularity of the partition.
    """
    if method == 'girvan_newman':
        communities_generator = nx.algorithms.community.girvan_newman(g)
        top_level_communities = next(communities_generator)
        communities = [list(c) for c in top_level_communities]
        modularity = nx.algorithms.community.modularity(g, communities)
    elif method == 'louvain':
        partition = community_louvain.best_partition(g)
        communities = list(set(partition.values()))
        communities = [[nodes for nodes, community in partition.items() if community == com] for com in communities]
        modularity = community_louvain.modularity(partition, g)
    else:
        raise ValueError(f"Unknown method: {method}")
    return communities, modularity

In [ ]:
gB = nx.read_graphml('gB.graphml')
gD = nx.read_graphml('gD.graphml')
fB = nx.read_graphml('fB.graphml')
hB = nx.read_graphml('hB.graphml')

In [ ]:
#Question 1
shared_nodes_gB_fB = num_common_nodes(gB, fB)
shared_nodes_gB_hB = num_common_nodes(gB, hB)
print(shared_nodes_gB_fB)
print(shared_nodes_gB_hB)

In [ ]:
#Question 2
central_nodes_degree = get_k_most_central(gB_prime, 'degree', 25)
central_nodes_betweenness = get_k_most_central(gB_prime, 'betweenness', 25)
common_central_nodes = num_common_nodes(central_nodes_degree, central_nodes_betweenness)


In [ ]:
#Question 3
cliques_gB_prime, nodes_in_cliques_gB_prime = find_cliques(gB_prime, min_size_clique)
cliques_gD_prime, nodes_in_cliques_gD_prime = find_cliques(gD_prime, min_size_clique)


In [ ]:
#Question 5
communities, modularity = detect_communities(gD, 'louvain') # or 'girvan_newman'


In [ ]:
#Question 7
nx.shortest_path(gB, 'Young Dro', 'Travis Porter')